In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pyvi

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from pyvi import ViTokenizer

In [ ]:
data = pd.read_json('/content/drive/MyDrive/Colab Notebooks/sentiment-analysis/data/data_coment_tiki.json')

In [ ]:
data = data.rename(columns={"content": "comment"})

In [ ]:
np.random.seed(10)
remove_n = 25000
drop_indices = np.random.choice(data[data['rating'] == 5].index, remove_n, replace=False)
scaled_data = data.drop(drop_indices)

In [ ]:
contents = []
file_stars = []
file_comments = []
with open('/content/drive/MyDrive/Colab Notebooks/sentiment-analysis/data/train.txt', encoding='utf-8') as f:
    contents = f.readlines()
    for content in contents:
        file_stars.append(content[0])
        file_comments.append(content[2:-2])

dictionary = {}
dictionary['comment'] = file_comments
dictionary['rating'] = file_stars
_df = pd.DataFrame(data=dictionary)
scaled_data.append(_df)
scaled_data

In [ ]:
import re
def remove_html(txt):
    return re.sub(r'<[^>]*>', '', txt)
 
txt = "<p class=\"par\">This is an example</p>"
remove_html(txt)

In [ ]:
# Chuẩn hóa unicode sang chuẩn unicode dựng sẵn
def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic


dicchar = loaddicchar()


def covert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

In [ ]:
acronym_words = []
acronym_words_dict = []
acronym_words = open('/content/drive/MyDrive/Colab Notebooks/sentiment-analysis/data/acronym_word.txt','r', encoding='utf-8')
acronym_words = acronym_words.readlines()
for i in range(len(acronym_words)):
  acronym_words_split = acronym_words[i].split("\t")
  for j in range(len(acronym_words_split)):
    acronym_words_split[j] = re.sub("[\n\ufeff]",'',acronym_words_split[j])
  try:
    acronym_words_dict.append({acronym_words_split[0]:acronym_words_split[1]})
  except: 
    pass

acronym_words_dict[:5]

In [ ]:
# Sử dụng bộ từ điển để thay thế các kiểu gõ dấu và viết tắt
def replace_acronyms(text):
  text_list = text.split(" ");
  for i in range(len(text_list)):
    for j in range(len(acronym_words_dict)):
      key = list(acronym_words_dict[j].keys())[0]
      value = list(acronym_words_dict[j].values())[0]
      if text_list[i] == key:
        text_list[i] = value
  return " ".join(text_list)

In [ ]:
def standardize_data(row):
    # Xóa dấu chấm, phẩy, hỏi ở cuối câu
    row = re.sub(r"[\.,\?]+$-", "", row)

    # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, dấu gạch dưới ... trong câu
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ").replace("_", " ")
    
    # Xóa các kí tự đặc biệt
    row = re.sub("\W",' ', row) 

    # Xóa các đường link
    row = re.sub('https?://\S+|www\.\S+', ' ', row)

    # Xóa các số
    row = re.sub('\w*\d\w*', '', row)

    #Xóa các kí tự xuống dòng
    row = " ".join(re.sub("\n", " ", row).split())

    row = row.strip().lower()
    
    return row

In [ ]:
def remove_loop_char(text):
  text = re.sub(r'([A-Z])\1+', lambda m: m.group(1).upper(), str(text), flags=re.IGNORECASE)
  text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',text)
  return text

In [ ]:
"""**Tiền xử lý dữ liệu**"""
# Làm sạch dữ liệu
def text_prosessing(text):
    # Chuyển đổi thành chữ thường
    text = text.lower() 

    # Xóa các đoạn mã HTML
    text = remove_html(text)

    # chuyển unicode tổ hợp sang chuẩn unicode dựng sẵn
    text = covert_unicode(text)

    # Thay thế từ viết tắt
    text = replace_acronyms(text)

    # Xóa từ dừng
    # text = remove_stop_word(text)

    # Chuẩn hóa câu
    text = standardize_data(text)

    # Xử lý các từ viết trùng lắp
    text = remove_loop_char(text)

    # tách từ
    # text = sementation(text)

    return text

In [ ]:
# Apply vào data
scaled_data['comment'] = scaled_data['comment'].apply(text_prosessing)

In [ ]:
# scaled_data['rating'] = scaled_data["rating"].map({1:-1, 2:-1, 3:0, 4:1, 5:1})
X = scaled_data['comment']
y = scaled_data['rating']

In [ ]:
_data = scaled_data.copy()
# _data["rating"] = _data["rating"].map({1:-1, 2:-1, 3:0, 4:1, 5:1})
_data.to_csv("/content/drive/MyDrive/Colab Notebooks/sentiment analysis/data/processed_data.csv")